# REDUCEDHEATCARB analysis

This JupyterLabs notebook can be used for physics ifnormed machine learning analysis in the REDUCEDHEATCAB project.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!

## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import numpy as np
import pandas as pd

%load_ext autoreload

    
from tqdm.notebook import tqdm

from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

from plotter import Plot
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget


from nfh_utils import *

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

from rhc_analysis import Learner

rhc_preprocessed_poperties_file='rhc_preprocessed_properties_intv_5_min.parquet'
# rhc_preprocessed_poperties_file='rhc_preprocessed_properties_intv_1_min.parquet'
#rhc_preprocessed_poperties_file='rhc_heat_dist_preprocessed_properties.parquet'

home_data_file_path = "home_data.parquet"
boiler_returntemp_load_efficiency_file_path = "boiler_returntemp_load_efficiency.parquet"
boiler_returntemp_efficiency_file_path = "boiler_returntemp_efficiency.parquet"




### Reading preprocessed interpolated properties from a parquet file

In [ ]:
%%time

# Attempt to read the Parquet file
try:
    df_prep = pd.read_parquet(
        rhc_preprocessed_poperties_file, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFrames are properly sorted
if not df_prep.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prep = df_prep.sort_index()  

In [ ]:
print(f"df_prep.count().sum(): {df_prep.count().sum():_}")

In [ ]:
df_prep.info()

In [ ]:
print("[\n'","', \n'".join(sorted(df_prep.columns)),"'\n]")

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$',
    'W' : r'$W$',
    'W_m_2' : r'$W/m^{2}$',
    'm_s_1' : r'$m/s$'    
}

In [ ]:
# visuaize all input data
df_plot = df_prep

In [ ]:
list(df_plot.index.unique('id').dropna())

In [ ]:
df_plot.index.unique('id').dropna()

In [ ]:
# df_plot.loc[[401632]][[prop for prop in df_plot.columns.values if prop.split('__')[-1] in ('degC', 'W', '0', 'bool', 'ppm', 'W_m_2')]]

In [ ]:
#Plot all properties from all sources for all ids
#Plot.dataframe_preprocessed_plot(df_plot.loc[[401632]][[prop for prop in df_plot.columns.values if prop.split('__')[-1] in ('degC', 'ppm', 'W_m_2')]], units_to_mathtext)

# Calculate additional properties 
We may have to move some of these calculations to inside the GEKKO Python model code (e.g. for the what-if scenario simulation)

### Calculating electricity data

In [ ]:
%%time
df_prep['calculated_device_p1-reader_e_use__W'] = df_prep['device_p1-reader_e_use_hi__W'] + df_prep['device_p1-reader_e_use_lo__W']
df_prep['calculated_device_p1-reader_e_ret__W'] = df_prep['device_p1-reader_e_ret_hi__W'] + df_prep['device_p1-reader_e_ret_lo__W'] 


### Reading and calculating boiler data 

#### Reading home metadata

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_homes = pd.read_parquet(
        home_data_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
df_homes.info()

In [ ]:
df_homes


#### Reading boiler data

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_boiler_efficiency = pd.read_parquet(
        # boiler_returntemp_efficiency_file_path, 
        boiler_returntemp_load_efficiency_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
df_boiler_efficiency

#### Prepare lookup of momentary boiler efficiency from boiler-specific efficiency curves

In [ ]:
%%time
# Round return temperatures to whole degrees
df_prep['rounded_temp_ret__degC'] = df_prep['batch_import_remeha_temp_ret__degC'].round()

In [ ]:
df_prep.info()

In [ ]:
df_prep.index.dtypes

In [ ]:
df_prep.count()

In [ ]:
df_returntemp = df_prep[df_prep['batch_import_remeha_boiler_status_burning_ch__bool'] == True][['rounded_temp_ret__degC', 'batch_import_remeha_temp_ret__degC']]

In [ ]:
df_returntemp.describe()

In [ ]:
bins = int(df_returntemp['rounded_temp_ret__degC'].max()) + 1 - int(df_returntemp['rounded_temp_ret__degC'].min())
df_returntemp['rounded_temp_ret__degC'].to_frame().dropna(subset=['rounded_temp_ret__degC']).hist(bins=bins, alpha=0.5)  # Adjust `bins` to your preference
plt.xticks(range(int(df_returntemp['rounded_temp_ret__degC'].min()), 
                 int(df_returntemp['rounded_temp_ret__degC'].max()) + 1))
plt.title("Global Histogram of rounded_temp_ret__degC")
plt.xlabel("Temperature (°C)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
len(df_returntemp[df_returntemp['rounded_temp_ret__degC'] < 30]) / len(df_returntemp)

In [ ]:
Plot.nfh_property_per_id_boxplot(df_prep, filter_col='batch_import_remeha_boiler_status_burning_ch__bool', property_col='rounded_temp_ret__degC')

In [ ]:
df_prep.groupby(level='id').count().T

In [ ]:
pd.concat([df_prep.dropna().groupby(level='id')['batch_import_remeha_ch_set_fan_rotations_min__min_1'].unique(),
           df_prep.dropna().groupby(level='id')['batch_import_remeha_ch_set_fan_rotations_max__min_1'].unique()],
          axis=1).sort_values(by='batch_import_remeha_ch_set_fan_rotations_max__min_1', ascending=True)

In [ ]:
df_prep.groupby(level='id')['batch_import_remeha_ch_set_fan_rotations_max__min_1'].unique()

In [ ]:
# Merge df_prep with df_homes to get the brand_model for each id
df_prep = df_prep.reset_index().merge(df_homes, on='id').set_index(['id', 'timestamp'])

In [ ]:
df_prep.info()

In [ ]:
df_prep.batch_import_remeha_temp_in__degC.mean()

In [ ]:
df_prep.count()

In [ ]:
remeha2nfh = {
    # 'parHeFanRpmChMax': 'batch_import_remeha_ch_set_fan_rotations_max__min_1',
    # 'parHeFanRpmMin': 'batch_import_remeha_ch_set_fan_rotations_min__min_1',
    # 'parHePowerMax': 'power_ch_max__kW',
    # 'parHePowerMin': 'power_ch_min__kW',
    # 'parHeFanRpmMin.Minimum': 'min_fan_rotations__min_1',
    # 'parHeFanRpmChMax.Maximum': 'max_ch_fan_rotations__min_1',
    'varHeFanRpm': 'batch_import_remeha_fan_rotations__min_1',
    'varApPowerActual': 'batch_import_remeha_g_use_ch_and_dhw__kW0'
}


In [ ]:
df_prep[df_prep['batch_import_remeha_boiler_status_burning_ch__bool'] == True][remeha2nfh['varHeFanRpm']].describe()

In [ ]:
%%time
# Create lookup value for boiler load fraction (how far along the current fan RPM is between the min and max)
df_prep.loc[:,'calculated_fan_frac__0'] = (
    (df_prep[remeha2nfh['varHeFanRpm']] - df_prep['fan_min_ch_rotations_min_1'])
    / 
    (df_prep['fan_max_ch_rotations__min_1'] - df_prep['fan_min_ch_rotations_min_1'])
).astype('Float32')

df_prep.loc[:,'rounded_load__%'] = (
    (df_prep['calculated_fan_frac__0'] + (1 - df_prep['calculated_fan_frac__0']) * df_prep['Qnh_min_lhv__kW'] / df_prep['Qnh_max_lhv__kW'])
    * 100
).round().astype('Int16')

In [ ]:
df_prep[df_prep['batch_import_remeha_boiler_status_burning_ch__bool'] == True]['calculated_fan_frac__0'].describe()

In [ ]:
df_prep[df_prep['batch_import_remeha_boiler_status_burning_ch__bool'] == True]['rounded_load__%'].describe()

In [ ]:
%%time                                         

df_prep.loc[:,'calculated_g_use_ch__m3_s_1'] = df_prep['batch_import_remeha_fan_rotations__min_1']  * df_prep['rotation__m3'] / s_min_1

# # initially, assume higher heating value of groningen gas
# df_prep.loc[:,'gas_std_hhv__J_m_3'] = gas_groningen_nl_avg_std_hhv__J_m_3

# alternatively, use average of actual higher heating value
df_prep.loc[:,'gas_std_hhv__J_m_3'] = 35.974632 * 1e6

# # alternatively, use the actual higher heating value
# df_prep.loc[:,'gas_std_hhv__J_m_3'] = df_prep.loc['actual_gas_std_hhv__J_m_3']

# estimate of the higher heating value of 1 m3 of the gas input 
df_prep.loc[:,'gas_hhv__J_m_3'] = (df_prep['gas_std_hhv__J_m_3'] 
                                   * (df_prep['batch_import_KNMI_air__Pa'] + overpressure_gas_nl_avg__Pa)
                                   / (P_std__Pa + overpressure_gas_nl_avg__Pa)
                                   ) * (temp_gas_std__K / temp_gas_avg_nl__K)

df_prep.loc[:,'calculated_g_use_ch_hhv__W'] = df_prep['calculated_g_use_ch__m3_s_1'] * df_prep['gas_hhv__J_m_3']

In [ ]:
# %%time                                         

# # calculate the gas input power
# df_prep.loc[:,'calculated_g_use_ch_lhv__W'] = 1e3 * df_prep['100%_load_ch__g25_3_ref_lhv__kW'] * df_prep['calculated_p_load__0']

# # Convert lower heating value of reference gas G25.3 used to Gronings gas, using higher heating value
# df_prep.loc[:,'calculated_g_use_ch_hhv__W'] = df_prep['calculated_g_use_ch_lhv__W'] /  gas_g25_3_ref_lhv__J__m3 * gas_groningen_nl_avg_std_hhv__J_m_3

In [ ]:
# Set the calculated gas use for central heating to zero when the boiler was not burning for central heating
df_prep.loc[df_prep['batch_import_remeha_boiler_status_burning_ch__bool'] == False, 'calculated_g_use_ch_hhv__W'] = 0

In [ ]:
%%time
#Store rounded values for lookup in efficiency tables
df_prep.loc[:,'rounded_temp_ret__degC'] = df_prep['batch_import_remeha_temp_ret__degC'].round().astype('Int8')


In [ ]:
Plot.nfh_property_per_id_boxplot(df_prep, filter_col='batch_import_remeha_boiler_status_burning_ch__bool', property_col='rounded_load__%')

In [ ]:
df_boiler_efficiency

#### Actual lookup of momentary boiler efficiency from boiler-specific efficiency curves

In [ ]:
%%time
# Merging DataFrames
df_prep = (df_prep
           .reset_index()
           .merge(df_boiler_efficiency.reset_index(),
                  on=['brand_model', 'rounded_load__%', 'rounded_temp_ret__degC'], 
                  how='left'
                  )
           .set_index(['id', 'timestamp'])
           .sort_index()
          )  


#### Calculating W_gain_ch__W based on boiler-specific efficiency

In [ ]:
%%time
df_prep.loc[:,'calculated_Q_gain_ch__W'] = df_prep['calculated_g_use_ch_hhv__W'] * df_prep['eta_ch_hhv__W0']

In [ ]:
list(df_prep.columns)

#### Inspect results

In [ ]:
df_heat_dist = df_prep[['batch_import_KNMI_ghi__W_m_2',
                        'batch_import_KNMI_temp_out__degC',
                        'batch_import_KNMI_wind__m_s_1',
                        'device_p1-reader_g_use_hhv__W',
                        'batch_import_remeha_boiler_status_burning_ch__bool',
                        'batch_import_remeha_gas_valve_closed__bool',
                        'batch_import_remeha_gas_valve_open__bool',
                        'batch_import_remeha_fan_rotations__min_1', 
                        'batch_import_remeha_g_use_ch_lhv__W',
                        'batch_import_remeha_temp_set__degC',
                        'batch_import_remeha_temp_in__degC',
                        'device_living_room_calibrated_temp_in__degC',
                        'batch_import_remeha_temp_sup__degC',
                        'batch_import_remeha_temp_ret__degC',
                        'batch_import_remeha_temp_ch_sup_max__degC', 
                        'calculated_fan_frac__0',
                        'calculated_g_use_ch_hhv__W',
                        'eta_ch_hhv__W0',
                        'calculated_Q_gain_ch__W',
                        'rounded_load__%',
                        'rounded_temp_ret__degC',
                       ]]

In [ ]:
df_heat_dist[df_heat_dist['batch_import_remeha_boiler_status_burning_ch__bool'] == True].groupby(level='id').agg({
    'batch_import_remeha_fan_rotations__min_1': ['min', 'mean', 'max'],
    'batch_import_remeha_g_use_ch_lhv__W': ['min', 'mean', 'max'],
    'calculated_fan_frac__0': ['min', 'mean', 'max'],
    'rounded_load__%': ['min', 'mean', 'max'],
    'calculated_g_use_ch_hhv__W': ['min', 'mean', 'max'],
    'calculated_Q_gain_ch__W': ['min', 'mean', 'max'],
}).T

In [ ]:
df_heat_dist.info()

In [ ]:
# Group by 'id' and calculate the mean for both 'eta_ch_hhv__W0' and 'batch_import_remeha_temp_ret__degC'
df_heat_dist[df_heat_dist['batch_import_remeha_boiler_status_burning_ch__bool'] == True].groupby(level='id').agg({
    'eta_ch_hhv__W0': 'mean',
    'batch_import_remeha_temp_ret__degC': 'mean',
    'batch_import_remeha_temp_ch_sup_max__degC': 'mean',
    'batch_import_remeha_temp_sup__degC': 'mean'
}).sort_values(by='eta_ch_hhv__W0', ascending=False)

In [ ]:
df_heat_dist[df_heat_dist['batch_import_remeha_boiler_status_burning_ch__bool'] == True]['eta_ch_hhv__W0'].mean() 

In [ ]:
Plot.nfh_property_per_id_boxplot(df_heat_dist, filter_col='batch_import_remeha_boiler_status_burning_ch__bool', property_col='eta_ch_hhv__W0')

## Learn energy profile parameters

Most of the heavy lifting is done by the `learn_energy_profile()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
# set room metadata to None, then learn_home_parameters() will derive the metadata from the ids.

hints = {
    'A_sol__m2': A_sol_nl_avg__m2,                      # initial estimate for apparent solar aperture
    'eta_ch_hhv__W0' : eta_ch_nl_avg_hhv__W0,           # average home heating efficiency of a gas boiler (based on higher heating value)
    'eta_not_ch_hhv__W0' : eta_not_ch_nl_avg_hhv__W0,   # average home heating efficiency indirecly DHW & cooking (based on higher heating value value)  
    'g_not_ch_hhv__W' : g_not_ch_nl_avg_hhv__W,         # average gas power (heating value) for other purposes than home heating [W] 
    'occupancy__p' : occupancy_nl_avg__p,               # average house occupancy (1.5: 2.2 persons, at home for 16.3 of 24 hours)
    'Q_gain_int__W_p_1' : Q_gain_int_nl_avg__W_p_1,     # average heat gain per occupant (77 W for 8.6 hours, 105 W for 7.7 hours)
    'wind_chill__K_s_m_1' : wind_chill_nl_avg__K_s_m_1, # average for Dutch homes, according to KNMI: https://cdn.knmi.nl/knmi/pdf/bibliotheek/knmipubmetnummer/knmipub219.pdf 
    'A_inf__m2': A_inf_nl_avg__m2,                       # average effective infiltration area corresponding to wind_chill_nl_avg__K_s_m_1
    'H_nl_avg__W_K_1' : H_nl_avg__W_K_1                 # average specific heat loss for a Dutch home
}

learn = ['A_sol__m2']

#define and select column names
property_sources = {
    'temp_in__degC' : 'batch_import_remeha_temp_in__degC',
    'temp_out__degC' : 'batch_import_KNMI_temp_out__degC',
    'wind__m_s_1' : 'batch_import_KNMI_wind__m_s_1',
    'ghi__W_m_2' : 'batch_import_KNMI_ghi__W_m_2', 
    'g_use_hhv__W' : 'device_p1-reader_g_use_hhv__W',
    'e_use__W' : 'calculated_device_p1-reader_e_use__W',
    'e_ret__W' : 'calculated_device_p1-reader_e_ret__W',
    'occupancy__p': 'device_living_room_occupancy__p' 
}

In [ ]:
# learn the model parameters and write results to a dataframe
%autoreload 2
df_results_per_period, df_results = Learner.learn_energy_profile(df_prep, 
                                                                  property_sources = property_sources, 
                                                                  learn = learn, 
                                                                  hints = hints,
                                                                  ev_type = 2
                                                                 )

### Result per learning period

In [ ]:
df_results_per_period

In [ ]:
# show essential statistics for the learned values
df_stats = df_results_per_period.describe().filter(regex='^actual_|^learned_')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the errors
df_stats = df_results_per_period.describe().filter(regex='^mae_|^rmse')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the error values, per id
df_stats = df_results_per_period.groupby('id').describe().stack().filter(regex='^mae_|^rmse')
df_stats.loc[df_stats.index.get_level_values(1).isin(['mean', 'std', 'min', 'max'])]

### Result Visualization

In [ ]:
df_results

In [ ]:
df_plot = df_prep[[prop for prop in df_prep.columns.values if prop.split('__')[-1] == 'degC']]

In [ ]:
#Plot only temperatures from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

In [ ]:
df_lot = df_prep

In [ ]:
#Plot all properties from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)